In [ ]:
!pip install git+https://github.com/Zahlii/shap.git

  Cloning https://github.com/Zahlii/shap.git to /tmp/pip-req-build-lpk596aq
  Running command git clone -q https://github.com/Zahlii/shap.git /tmp/pip-req-build-lpk596aq
  Created wheel for shap: filename=shap-0.37.0-cp36-cp36m-linux_x86_64.whl size=460211 sha256=471d69e79009f1b5690980f6f221e89bb1cb5dbac6f41c09797bdea13a3a0528
  Stored in directory: /tmp/pip-ephem-wheel-cache-_59a74am/wheels/7f/d4/9f/7bf00562d7a4742c6657a355abf79c8ed32f82dfc7d2b02d52
Successfully built shap


In [ ]:
import tensorflow as tf
from __future__ import print_function
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb

In [ ]:
tf.__version__

'2.4.0'

In [ ]:
max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Loading data...
17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)


In [ ]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128, input_shape=(x_train.shape[1],)))
model.add(LSTM(64))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size, epochs=3)
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Build model...
Train...
Epoch 1/3
782/782 [==============================] - 61s 76ms/step - loss: 0.5130 - accuracy: 0.7291
Epoch 2/3
782/782 [==============================] - 62s 79ms/step - loss: 0.2277 - accuracy: 0.9108
Epoch 3/3
782/782 [==============================] - 9s 12ms/step - loss: 0.4301 - accuracy: 0.8207
Test score: 0.43012136220932007
Test accuracy: 0.8207200169563293


In [ ]:
import shap

# we use the first 100 training examples as our background dataset to integrate over
explainer = shap.DeepExplainer(model, x_train[:100])

# explain the first 10 predictions
# explaining each prediction requires 2 * background dataset size runs
shap_values = explainer.shap_values(x_test[:10])

`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


InternalError: ignored

In [ ]:
# transform the indexes to words
import numpy as np
words = imdb.get_word_index()
num2word = {}
for w in words.keys():
    num2word[words[w]] = w
x_test_words = np.stack([np.array(list(map(lambda x: num2word.get(x, "NONE"), x_test[i]))) for i in range(10)])

In [ ]:
# plot the explanation of the first prediction
# Note the model is "multi-output" because it is rank-2 but only has one column
# init the JS visualization code
shap.initjs()
shap.force_plot(explainer.expected_value[0], shap_values[0][0], x_test_words[0])